# Linear Regression Model

### Intent

This model is not intended to be a classifier or predictor.

It will be trained on data where each row represents a flare up. 

We will look at how the model weigths the attributes, in order to know which attributes are most relevant, thus guiding data processing.

### Expected Input

The attributes and the flare up indicator for the n previous days (ie. for days t-1, t-2, ..., t-n).

### Expected Output

The flare up indicator for one disease on day t.

### Steps

Data preparation:

1. Create a function to prepare the input array (consider window width = w)
2. One row per flare up
3. Prepare the output array
4. Split into train and test set
5. Normalize data (fit the scaler to the train set only)

Model

- Linear regression
- Lasso

## Setup

In [9]:
import pandas as pd
import numpy as np
import copy
from sklearn import linear_model, preprocessing
import matplotlib.pyplot as plt
import pickle

In [10]:
# Connect to the s3 bucket
from sagemaker import get_execution_role
import boto3
bucket = 'montrealgr2'
role = get_execution_role()

In [11]:
# Read the dataset (binary output) from s3 bucket
data_key = 'data_min_5days.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
data_binary_out = pd.read_csv(data_location, low_memory = False)

In [12]:
# Read the dataset (0-4 output) from s3 bucket
data_key = 'data_min_5days_nonbinary.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
data_nonbinary_out = pd.read_csv(data_location, low_memory = False)

## Data Preparation

### Keep only numerical values in the dataset (so drop columns or do one_hot_encoding

Functions to prepare the dataset for a supervised learning task

In [13]:
def one_hot(dataframe, col_name):
    """
    (pandas.DataFrame, str) -> pandas.DataFrame
    
    Returns a DataFrame in which the column col_name has been one_hot encoded.
    The new columns are added to the left of the DataFrame.
    """
    # Create the new columns
    one_hot_df = pd.get_dummies(dataframe[col_name])
    # Drop the old column
    return_df = dataframe.drop(col_name,axis = 1)
    # Put the DataFrames together
    return_df = one_hot_df.join(return_df)
    
    return return_df

In [14]:
def prep_data(dataframe, icon=True, country=False, sex=True):
    """
    (pandas.DataFrame, bool, bool, bool) -> pandas.DataFrame
    
    Returns a DataFrame that contains only numerical values.
    The input dataframe is expected to have string values in these columns: "country", "icon", and "gender"
    
    country: True means one-hot encode the country column. False means drop the country column
    icon: True means one-hot encode the country column. False means drop the country column
    gender: True means one-hot encode the country column. False means drop the country column
    
    """
    
    numerical_df = dataframe.copy()
    
    # icon
    if icon == True:
        numerical_df = one_hot(numerical_df, "icon")
    else:
        numerical_df.drop(["icon"], axis=1, inplace=True)
        
    # country
    if country == True:
        numerical_df = one_hot(numerical_df, "country")
    else:
        numerical_df.drop(["country"], axis=1, inplace=True)
        
    # sex
    if sex == True:
        numerical_df = one_hot(numerical_df, "sex")
    else:
        numerical_df.drop(["sex"], axis=1, inplace=True)
        
    return numerical_df

Modify the dataset

In [15]:
data_binary_out = prep_data(data_binary_out)
data_binary_out

doesnt_say  female  male  other  clear-day  cloudy  fog  \
0                0       1     0      0          0       0    0   
1                0       1     0      0          0       0    0   
2                0       1     0      0          0       0    0   
3                0       1     0      0          0       0    0   
4                0       1     0      0          0       0    0   
...            ...     ...   ...    ...        ...     ...  ...   
139659           0       0     1      0          0       0    0   
139660           0       0     1      0          0       0    0   
139661           0       0     1      0          0       0    0   
139662           0       0     1      0          1       0    0   
139663           0       0     1      0          0       0    0   

        partly-cloudy-day  partly-cloudy-night  rain  ...  tiredness_lvl  IBS  \
0                       0                    0     0  ...            1.0  NaN   
1                       0                    0     0  ...            1.0  NaN   
2                       0                    0     0  ...            1.0  NaN   
3                       0                    0     0  ...            1.0  NaN   
4                       0                    0     0  ...            1.0  NaN   
...                   ...                  ...   ...  ...            ...  ...   
139659                  0                    1     0  ...            1.0  NaN   
139660                  1                    0     0  ...            1.0  NaN   
139661                  1                    0     0  ...            1.0  NaN   
139662                  0                    0     0  ...            1.0  NaN   
139663                  0                    1     0  ...            1.0  NaN   

        Lupus  Rheumatoid Arthritis  Fibromyalgia  Dysautonomia  \
0         NaN                   1.0           1.0           NaN   
1         NaN                   1.0           1.0           NaN   
2         NaN                   1.0           1.0           NaN   
3         NaN                   1.0           1.0           NaN   
4         NaN                   1.0           1.0           NaN   
...       ...                   ...           ...           ...   
139659    NaN                   NaN           1.0           NaN   
139660    NaN                   NaN           1.0           NaN   
139661    NaN                   NaN           1.0           NaN   
139662    NaN                   NaN           1.0           NaN   
139663    NaN                   NaN           1.0           NaN   

        Psoriatic arthritis  Arthritis  Psoriasis  Raynaud's Phenomenon  
0                       NaN        NaN        NaN                   NaN  
1                       NaN        NaN        NaN                   NaN  
2                       NaN        NaN        NaN                   NaN  
3                       NaN        NaN        NaN                   NaN  
4                       NaN        NaN        NaN                   NaN  
...                     ...        ...        ...                   ...  
139659                  NaN        NaN        NaN                   NaN  
139660                  NaN        NaN        NaN                   NaN  
139661                  NaN        NaN        NaN                   NaN  
139662                  NaN        NaN        NaN                   NaN  
139663                  NaN        NaN        NaN                   NaN  

[139664 rows x 34 columns]

In [16]:
data_nonbinary_out = prep_data(data_nonbinary_out)
data_nonbinary_out

doesnt_say  female  male  other  clear-day  cloudy  fog  \
0                0       1     0      0          0       0    0   
1                0       1     0      0          0       0    0   
2                0       1     0      0          0       0    0   
3                0       1     0      0          0       0    0   
4                0       1     0      0          0       0    0   
...            ...     ...   ...    ...        ...     ...  ...   
139659           0       0     1      0          0       0    0   
139660           0       0     1      0          0       0    0   
139661           0       0     1      0          0       0    0   
139662           0       0     1      0          1       0    0   
139663           0       0     1      0          0       0    0   

        partly-cloudy-day  partly-cloudy-night  rain  ...  tiredness_lvl  IBS  \
0                       0                    0     0  ...            1.0  NaN   
1                       0                    0     0  ...            1.0  NaN   
2                       0                    0     0  ...            1.0  NaN   
3                       0                    0     0  ...            1.0  NaN   
4                       0                    0     0  ...            1.0  NaN   
...                   ...                  ...   ...  ...            ...  ...   
139659                  0                    1     0  ...            1.0  NaN   
139660                  1                    0     0  ...            1.0  NaN   
139661                  1                    0     0  ...            1.0  NaN   
139662                  0                    0     0  ...            1.0  NaN   
139663                  0                    1     0  ...            1.0  NaN   

        Lupus  Rheumatoid Arthritis  Fibromyalgia  Dysautonomia  \
0         NaN                   2.0           3.0           NaN   
1         NaN                   3.0           3.0           NaN   
2         NaN                   3.0           3.0           NaN   
3         NaN                   3.0           3.0           NaN   
4         NaN                   2.0           3.0           NaN   
...       ...                   ...           ...           ...   
139659    NaN                   NaN           4.0           NaN   
139660    NaN                   NaN           4.0           NaN   
139661    NaN                   NaN           4.0           NaN   
139662    NaN                   NaN           4.0           NaN   
139663    NaN                   NaN           4.0           NaN   

        Psoriatic arthritis  Arthritis  Psoriasis  Raynaud's Phenomenon  
0                       NaN        NaN        NaN                   NaN  
1                       NaN        NaN        NaN                   NaN  
2                       NaN        NaN        NaN                   NaN  
3                       NaN        NaN        NaN                   NaN  
4                       NaN        NaN        NaN                   NaN  
...                     ...        ...        ...                   ...  
139659                  NaN        NaN        NaN                   NaN  
139660                  NaN        NaN        NaN                   NaN  
139661                  NaN        NaN        NaN                   NaN  
139662                  NaN        NaN        NaN                   NaN  
139663                  NaN        NaN        NaN                   NaN  

[139664 rows x 34 columns]

### Only keep data for one disease

A function that creates a dataset that contains only one condition

In [17]:
# Lists of conditions
all_cond = ["IBS", "Lupus", "Rheumatoid Arthritis", "Fibromyalgia", "Dysautonomia", 
            "Psoriatic arthritis", "Arthritis", "Psoriasis", "Raynaud's Phenomenon"]

In [18]:
def keep_one_cond(dataframe, cond, cond_list):
    """
    (pandas.DataFrame, string, list) -> pandas.DataFrame
    
    The function keeps all the entries of any user who has the given cond.
    Users are said to have a condition if they reported a flare up intensity >= 1 at least once for the given cond.
    
    """
    # List the irrelevant conditions
    irrelevant_cond = copy.copy(cond_list)
    irrelevant_cond.remove(cond)
    
    # All the user_ids that have at least one flare up (>=1) for cond
    cond_users = dataframe[dataframe.loc[:,(cond)]>=1]["user_id"].unique()
    # Dataframe containing only and all the data of people who have at least one flare up
    cond_df = dataframe[dataframe.loc[:,("user_id")].isin(cond_users)]
    # Drop columns for irrelevant conditions
    #cond_df.drop(labels=irrelevant_cond, axis=1, inplace=True) <---Dongyan changed this
    cond_df = cond_df.drop(labels=irrelevant_cond, axis=1, inplace=False)  # <----to this
    
    return cond_df

Create a dataframe for each condition

In [19]:
# Dataframes with binary flare up indicators
# The names of the dataframes that will be created:

pd.set_option('mode.chained_assignment', 'raise') # <---Dongyan's note: changing 'raise' to None ignores the warning

binary_df_names = ['IBS_binary', "lupus_binary", "rheum_arthr_binary", "fibromyalgia_binary", "dysautonomia_binary", 
                  "psor_arthr_binary", "arthr_binary", "psor_binary", "raynauds_binary"]
# Dictionary to store the dataframes with their name:
binary_df_dict = {}
for i, cond in enumerate(all_cond):
    binary_df_dict[binary_df_names[i]] = keep_one_cond(data_binary_out, cond, all_cond)

# Dataframes with NONbinary flare up indicators
# The names of the dataframes that will be created:
nonbinary_df_names = ['IBS_nonbinary', "lupus_nonbinary", "rheum_arthr_nonbinary", "fibromyalgia_nonbinary", "dysautonomia_nonbinary", 
                  "psor_arthr_nonbinary", "arthr_nonbinary", "psor_nonbinary", "raynauds_nonbinary"]
# Dictionary to store the dataframes with their name:
nonbinary_df_dict = {}
for i, cond in enumerate(all_cond):
    nonbinary_df_dict[nonbinary_df_names[i]]= keep_one_cond(data_nonbinary_out, cond, all_cond)


### Deal with NaN values

Option 1) Drop all rows with NaN values

In [20]:
# Dataframes with binary flare up indicators
# Dictionary to store the new dataframes with their name:
dropna_binary_df_dict = {}
dropna_binary_df_names = ['IBS_dropna_binary', "lupus_dropna_binary", "rheum_arthr_dropna_binary", 
                          "fibromyalgia_dropna_binary", "dysautonomia_dropna_binary", "psor_arthr_dropna_binary", 
                          "arthr_dropna_binary", "psor_dropna_binary", "raynauds_dropna_binary"]
# Store each name as a key pointing to the corresponding dataframe (which contains only rows without any NaN values)
for i, binary_df_name in enumerate (binary_df_names):
    new_df = binary_df_dict[binary_df_name].dropna()
    dropna_binary_df_dict[dropna_binary_df_names[i]] = new_df

# Dataframes with non binary flare up indicators
# Dictionary to store the new dataframes with their name:
dropna_nonbinary_df_dict = {}
dropna_nonbinary_df_names = ['IBS_dropna_nonbinary', "lupus_dropna_nonbinary", "rheum_arthr_dropna_nonbinary", 
                          "fibromyalgia_dropna_nonbinary", "dysautonomia_dropna_nonbinary", "psor_arthr_dropna_nonbinary", 
                          "arthr_dropna_nonbinary", "psor_dropna_nonbinary", "raynauds_dropna_nonbinary"]
# Store each name as a key pointing to the corresponding dataframe (which contains only rows without any NaN values)
for i, nonbinary_df_name in enumerate (nonbinary_df_names):
    new_df = nonbinary_df_dict[nonbinary_df_name].dropna()
    dropna_nonbinary_df_dict[dropna_nonbinary_df_names[i]] = new_df

Option 2) Replace some of the NaN values

In [21]:
def replace_na(df):
    """
    (pandas.DataFrame) -> pandas.DataFrame
    
    Replaces some NaN values of df with appropriate filler values.
    Then drops all the rows that still contain an NaN value.
    """
    
    # Drop null weather rows
    df = df[df['humidity'].notna()].copy() # <--- Dongyan added .copy()
    
    # Fill in other NaN values with appropriate filler
    df['precip_intensity'] = df['precip_intensity'].fillna(0)
    df['exercise_lvl'] = df['exercise_lvl'].fillna(1)
    df['tiredness_lvl'] = df['tiredness_lvl'].fillna(0)
    df['depression_lvl'] = df['depression_lvl'].fillna(0)
    df['stress_lvl'] = df['stress_lvl'].fillna(0)
    
    # Fill out NaN conditions with 0
    df = df.fillna(0)
    
    # Drop other rows that still contain NaN values
    df = df.dropna()
    
    return df

In [22]:
# Dataframes with binary flare up indicators
# Dictionary to store the new dataframes with their name:
repna_binary_df_dict = {}
repna_binary_df_names = ['IBS_repna_binary', "lupus_repna_binary", "rheum_arthr_repna_binary", 
                          "fibromyalgia_repna_binary", "dysautonomia_repna_binary", "psor_arthr_repna_binary", 
                          "arthr_repna_binary", "psor_repna_binary", "raynauds_repna_binary"]
# Store each name as a key pointing to the corresponding dataframe (which contains only rows without any NaN values)
for i, binary_df_name in enumerate (binary_df_names):
    old_df = binary_df_dict[binary_df_name]
    new_df = replace_na(old_df)
    repna_binary_df_dict[repna_binary_df_names[i]] = new_df

# Dataframes with non binary flare up indicators
# Dictionary to store the new dataframes with their name:
repna_nonbinary_df_dict = {}
repna_nonbinary_df_names = ['IBS_repna_nonbinary', "lupus_repna_nonbinary", "rheum_arthr_repna_nonbinary",
                          "fibromyalgia_repna_nonbinary", "dysautonomia_repna_nonbinary", "psor_arthr_repna_nonbinary", 
                          "arthr_repna_nonbinary", "psor_repna_nonbinary", "raynauds_repna_nonbinary"]
# Store each name as a key pointing to the corresponding dataframe (which contains only rows without any NaN values)
for i, nonbinary_df_name in enumerate (nonbinary_df_names):
    old_df = nonbinary_df_dict[nonbinary_df_name]
    new_df = replace_na(old_df)
    repna_nonbinary_df_dict[repna_nonbinary_df_names[i]] = new_df

_Note_: now we have the following dictionaries in which the keys are strings that map to a dataframe:

- dropna_binary_df_dict (NaN values dropped, flare value 0/1)
- dropna_nonbinary_df_dict (NaN values dropped, flare value 0/1/2/3/4)
- repna_binary_df_dict (NaN values partly replaced, flare value 0/1)
- repna_nonbinary_df_dict (NaN values partly replaced, flare value 0/1/2/3/4)

These dataframes still need to be modified before being fed to a model.

### A function that prepares the input array such as to have many days' information in the input

In [23]:
def sliding_window_data (dataframe, window_width):
    """"
    (pandas.DataFrame, int) -> pandas.DataFrame
    
    Converts dataframe into a supervised learning dataset.
    The input in the dataset considers n time steps previous to time t.
    
    dataframe: a dataframe in which the last column is the output value
    window_width: the number of timesteps included in the input
    
    inspired from: https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
    
    >>> test_df = pd.DataFrame()
    >>> test_df["Stress"] = [0, 1, 1, 0]
    >>> test_df["Exercise"] = [1, 4, 3, 1]
    >>> test_df["Flare_up"] = [0, 0, 1, 1]
    >>> print (test_df)
           Stress  Exercise  Flare_up
    0       0         1         0
    1       1         4         0
    2       1         3         1
    3       0         1         1

    
    >>> test_window_df = sliding_window_data(test_df, 3)
    >>> test_window_df
           Stress_(t-3)  Exercise_(t-3)  Flare_up_(t-3)  Stress_(t-2)  Exercise_(t-2)  \
    0           NaN             NaN             NaN           NaN             NaN   
    1           NaN             NaN             NaN           NaN             NaN   
    2           NaN             NaN             NaN           0.0             1.0   
    3           0.0             1.0             0.0           1.0             4.0   

       Flare_up_(t-2)  Stress_(t-1)  Exercise_(t-1)  Flare_up_(t-1)  label  
    0             NaN           NaN             NaN             NaN      0  
    1             NaN           0.0             1.0             0.0      0  
    2             0.0           1.0             4.0             0.0      1  
    3             0.0           1.0             3.0             1.0      1  
    
    """
    
    var_names = list(dataframe.columns)
    n_vars = dataframe.shape[1]
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(window_width, 0, -1):
        cols.append(dataframe.shift(i))
        names += [('%s_(t-%d)' % (var_names[j], i)) for j in range(n_vars)]
    
    # output value
    cols.append(dataframe.iloc[:,-1:])
    names.append("label")
    
    # Put it all together
    window_data = pd.concat(cols, axis=1)
    window_data.columns = names
    return window_data

### Keep the long enough sequence of days and arrange them in a sliding window dataset

In [24]:
def prep_dataset(dataframe, window_width):
    """
    (pandas.DataFrame, int) -> pandas.DataFrame
    
    Returns a DataFrame in which we keep only the users who have enough uninterrupted entries to account for the window_width.
    window_width: the number of days past that we want to give as input to the model. 
    The minimum number of entries required = window_width + 1

    """
    
    # Iterate through the users in the dataframe to keep the users with enough uninterrupted login days
    users_rows_dict = dict(dataframe["user_id"].value_counts())
    users_uninter_days_dict = {} # user_id (as key) -> list of list (sequences of uninterrupted login days) (as value)
    for user in users_rows_dict:
        num_rows = users_rows_dict[user]
        login_days = [] # a list to store lists of uninterrupted sequences of days on which a user provided data
        for i, row in dataframe.iterrows():
            checkin_date = dataframe.loc[i, "checkin_date"]
            # If this date does not immediately follow the previous date, use this data as the start of a new sequence of login days
            if (len(login_days) == 0) or (checkin_date != login_days[-1][-1] + 1):
                new_seq = []
                new_seq.append(checkin_date)
                login_days.append(new_seq)
            # If this date immediately follows the provious checkin_date, add this date to the existing sequence of uninterrupted login days
            else:
                login_days[-1].append(checkin_date)
        # Only keep the sequences of days that are long enough
        good_login_days = []
        for seq in login_days:
            if len(seq) > window_width:
                good_login_days.append(seq)
        # Store the long sequences of days in users_uninter_days_dict
        if len(good_login_days) != 0:
            users_uninter_days_dict[user]=good_login_days
    
    # Keep only the users with long enough login sequences and keep only the days that are part of a long enough login sequence
    # Then create a dataframe in which the input is sequential (many days are included in the input)
    good_window_df = pd.DataFrame()
    
    for user in users_uninter_days_dict:
        one_usr_df = dataframe[dataframe.loc[:,"user_id"]==user]        
        # We will create a dataframe for each day sequence
        for seq in users_uninter_days_dict[user]:
            one_seq_df = one_usr_df[one_usr_df.loc[:,"checkin_date"].isin(seq)]
            one_seq_df = one_seq_df.drop(["user_id", 'checkin_date'], axis=1)
            # Put may days in the input
            one_seq_window = sliding_window_data(one_seq_df, window_width)
            # Drop rows with NAN
            one_seq_window = one_seq_window.dropna(axis=0)
            # Append this sequence's rows to all the other sequences' rows
            good_window_df = pd.concat([good_window_df, one_seq_window])
        
    return good_window_df


### Create the final DataFrames 

For each disease, for different relevant window widths (1,3,5,10)

In [25]:
def each_cond_dataset(old_df_dict, old_df_names, new_df_dict, new_df_names, window_width):
    """
    (dict, list, dict, list, int) -> NoneType
    
    Fills in new_df_dict with keys new_df_names mapping to a new DataFrame. 
    Those new DataFrames take the window_width into account and will be ready to be fed to a supervised learning model.
    One new DataFrame is created for each old DataFrame in old_df_dict
    
    """
    for i, old_name in enumerate (old_df_names):
        old_df = old_df_dict[old_name]
        new_df = prep_dataset(old_df, window_width)
        new_df_dict[new_df_names[i]] = new_df
    

Dataframes in which NaN values have been dropped and the flare up indicator is binary

In [26]:
# width: 1
# Dictionary to store the new dataframes with their name:
dropna_binary_w1_df_dict = {}
dropna_binary_w1_df_names = ['IBS_dropna_binary_w1', "lupus_dropna_binary_w1", "rheum_arthr_dropna_binary_w1", 
                          "fibromyalgia_dropna_binary_w1", "dysautonomia_dropna_binary_w1", "psor_arthr_dropna_binary_w1", 
                          "arthr_dropna_binary_w1", "psor_dropna_binary_w1", "raynauds_dropna_binary_w1"]
each_cond_dataset(dropna_binary_df_dict, dropna_binary_df_names, dropna_binary_w1_df_dict, dropna_binary_w1_df_names, 1)

In [27]:
# width: 3
dropna_binary_w3_df_dict = {}
dropna_binary_w3_df_names = ['IBS_dropna_binary_w3', "lupus_dropna_binary_w3", "rheum_arthr_dropna_binary_w3", 
                          "fibromyalgia_dropna_binary_w3", "dysautonomia_dropna_binary_w3", "psor_arthr_dropna_binary_w3", 
                          "arthr_dropna_binary_w3", "psor_dropna_binary_w3", "raynauds_dropna_binary_w3"]
each_cond_dataset(dropna_binary_df_dict, dropna_binary_df_names, dropna_binary_w3_df_dict, dropna_binary_w3_df_names, 3)

In [28]:
# width: 5
dropna_binary_w5_df_dict = {}
dropna_binary_w5_df_names = ['IBS_dropna_binary_w5', "lupus_dropna_binary_w5", "rheum_arthr_dropna_binary_w5", 
                          "fibromyalgia_dropna_binary_w5", "dysautonomia_dropna_binary_w5", "psor_arthr_dropna_binary_w5", 
                          "arthr_dropna_binary_w5", "psor_dropna_binary_w5", "raynauds_dropna_binary_w5"]
each_cond_dataset(dropna_binary_df_dict, dropna_binary_df_names, dropna_binary_w5_df_dict, dropna_binary_w5_df_names, 5)

In [29]:
# width: 10
dropna_binary_w10_df_dict = {}
dropna_binary_w10_df_names = ['IBS_dropna_binary_w10', "lupus_dropna_binary_w10", "rheum_arthr_dropna_binary_w10", 
                          "fibromyalgia_dropna_binary_w10", "dysautonomia_dropna_binary_w10", "psor_arthr_dropna_binary_w10", 
                          "arthr_dropna_binary_w10", "psor_dropna_binary_w10", "raynauds_dropna_binary_w10"]
each_cond_dataset(dropna_binary_df_dict, dropna_binary_df_names, dropna_binary_w10_df_dict, dropna_binary_w10_df_names, 10)

Dataframes in which NaN values have been dropped and the flare up indicator is not binary (0-4)

In [30]:
# width: 1
dropna_nonbinary_w1_df_dict = {}
dropna_nonbinary_w1_df_names = ['IBS_dropna_nonbinary_w1', "lupus_dropna_nonbinary_w1", "rheum_arthr_dropna_nonbinary_w1", 
                          "fibromyalgia_dropna_nonbinary_w1", "dysautonomia_dropna_nonbinary_w1", "psor_arthr_dropna_nonbinary_w1", 
                          "arthr_dropna_nonbinary_w1", "psor_dropna_nonbinary_w1", "raynauds_dropna_nonbinary_w1"]
each_cond_dataset(dropna_nonbinary_df_dict, dropna_nonbinary_df_names, dropna_nonbinary_w1_df_dict, dropna_nonbinary_w1_df_names, 1)

In [31]:
# width: 3
dropna_nonbinary_w3_df_dict = {}
dropna_nonbinary_w3_df_names = ['IBS_dropna_nonbinary_w3', "lupus_dropna_nonbinary_w3", "rheum_arthr_dropna_nonbinary_w3", 
                          "fibromyalgia_dropna_nonbinary_w3", "dysautonomia_dropna_nonbinary_w3", "psor_arthr_dropna_nonbinary_w3", 
                          "arthr_dropna_nonbinary_w3", "psor_dropna_nonbinary_w3", "raynauds_dropna_nonbinary_w3"]
each_cond_dataset(dropna_nonbinary_df_dict, dropna_nonbinary_df_names, dropna_nonbinary_w3_df_dict, dropna_nonbinary_w3_df_names, 3)

In [32]:
# width: 5
dropna_nonbinary_w5_df_dict = {}
dropna_nonbinary_w5_df_names = ['IBS_dropna_nonbinary_w5', "lupus_dropna_nonbinary_w5", "rheum_arthr_dropna_nonbinary_w5", 
                          "fibromyalgia_dropna_nonbinary_w5", "dysautonomia_dropna_nonbinary_w5", "psor_arthr_dropna_nonbinary_w5", 
                          "arthr_dropna_nonbinary_w5", "psor_dropna_nonbinary_w5", "raynauds_dropna_nonbinary_w5"]
each_cond_dataset(dropna_nonbinary_df_dict, dropna_nonbinary_df_names, dropna_nonbinary_w5_df_dict, dropna_nonbinary_w5_df_names, 5)

In [33]:
# width: 10
dropna_nonbinary_w10_df_dict = {}
dropna_nonbinary_w10_df_names = ['IBS_dropna_nonbinary_w10', "lupus_dropna_nonbinary_w10", "rheum_arthr_dropna_nonbinary_w10", 
                          "fibromyalgia_dropna_nonbinary_w10", "dysautonomia_dropna_nonbinary_w10", "psor_arthr_dropna_nonbinary_w10", 
                          "arthr_dropna_nonbinary_w10", "psor_dropna_nonbinary_w10", "raynauds_dropna_nonbinary_w10"]
each_cond_dataset(dropna_nonbinary_df_dict, dropna_nonbinary_df_names, dropna_nonbinary_w10_df_dict, dropna_nonbinary_w10_df_names, 10)

Dataframes in which NaN values have been replaced

In [ ]:
# Dataframes with binary flare up indicators
# width: 1
# Dictionary to store the new dataframes with their name:
repna_binary_w1_df_dict = {}
repna_binary_w1_df_names = ['IBS_repna_binary_w1', "lupus_repna_binary_w1", "rheum_arthr_repna_binary_w1", 
                          "fibromyalgia_repna_binary_w1", "dysautonomia_repna_binary_w1", "psor_arthr_repna_binary_w1", 
                          "arthr_repna_binary_w1", "psor_repna_binary_w1", "raynauds_repna_binary_w1"]
each_cond_dataset(repna_binary_df_dict, repna_binary_df_names, repna_binary_w1_df_dict, repna_binary_w1_df_names, 1)

# width: 3
repna_binary_w3_df_dict = {}
repna_binary_w3_df_names = ['IBS_repna_binary_w3', "lupus_repna_binary_w3", "rheum_arthr_repna_binary_w3", 
                          "fibromyalgia_repna_binary_w3", "dysautonomia_repna_binary_w3", "psor_arthr_repna_binary_w3", 
                          "arthr_repna_binary_w3", "psor_repna_binary_w3", "raynauds_repna_binary_w3"]
each_cond_dataset(repna_binary_df_dict, repna_binary_df_names, repna_binary_w3_df_dict, repna_binary_w3_df_names, 3)

# width: 5
repna_binary_w5_df_dict = {}
repna_binary_w5_df_names = ['IBS_repna_binary_w5', "lupus_repna_binary_w5", "rheum_arthr_repna_binary_w5", 
                          "fibromyalgia_repna_binary_w5", "dysautonomia_repna_binary_w5", "psor_arthr_repna_binary_w5", 
                          "arthr_repna_binary_w5", "psor_repna_binary_w5", "raynauds_repna_binary_w5"]
each_cond_dataset(repna_binary_df_dict, repna_binary_df_names, repna_binary_w5_df_dict, repna_binary_w5_df_names, 5)

# width: 10
repna_binary_w10_df_dict = {}
repna_binary_w10_df_names = ['IBS_repna_binary_w10', "lupus_repna_binary_w10", "rheum_arthr_repna_binary_w10", 
                          "fibromyalgia_repna_binary_w10", "dysautonomia_repna_binary_w10", "psor_arthr_repna_binary_w10", 
                          "arthr_repna_binary_w10", "psor_repna_binary_w10", "raynauds_repna_binary_w10"]
each_cond_dataset(repna_binary_df_dict, repna_binary_df_names, repna_binary_w10_df_dict, repna_binary_w10_df_names, 10)


# Dataframes with non binary flare up indicators
# Dictionary to store the new dataframes with their name:
# width: 1
repna_nonbinary_w1_df_dict = {}
repna_nonbinary_w1_df_names = ['IBS_repna_nonbinary_w1', "lupus_repna_nonbinary_w1", "rheum_arthr_repna_nonbinary_w1", 
                          "fibromyalgia_repna_nonbinary_w1", "dysautonomia_repna_nonbinary_w1", "psor_arthr_repna_nonbinary_w1", 
                          "arthr_repna_nonbinary_w1", "psor_repna_nonbinary_w1", "raynauds_repna_nonbinary_w1"]
each_cond_dataset(repna_nonbinary_df_dict, repna_nonbinary_df_names, repna_nonbinary_w1_df_dict, repna_nonbinary_w1_df_names, 1)

# width: 3
repna_nonbinary_w3_df_dict = {}
repna_nonbinary_w3_df_names = ['IBS_repna_nonbinary_w3', "lupus_repna_nonbinary_w3", "rheum_arthr_repna_nonbinary_w3", 
                          "fibromyalgia_repna_nonbinary_w3", "dysautonomia_repna_nonbinary_w3", "psor_arthr_repna_nonbinary_w3", 
                          "arthr_repna_nonbinary_w3", "psor_repna_nonbinary_w3", "raynauds_repna_nonbinary_w3"]
each_cond_dataset(repna_nonbinary_df_dict, repna_nonbinary_df_names, repna_nonbinary_w3_df_dict, repna_nonbinary_w3_df_names, 3)

# width: 5
repna_nonbinary_w5_df_dict = {}
repna_nonbinary_w5_df_names = ['IBS_repna_nonbinary_w5', "lupus_repna_nonbinary_w5", "rheum_arthr_repna_nonbinary_w5", 
                          "fibromyalgia_repna_nonbinary_w5", "dysautonomia_repna_nonbinary_w5", "psor_arthr_repna_nonbinary_w5", 
                          "arthr_repna_nonbinary_w5", "psor_repna_nonbinary_w5", "raynauds_repna_nonbinary_w5"]
each_cond_dataset(repna_nonbinary_df_dict, repna_nonbinary_df_names, repna_nonbinary_w5_df_dict, repna_nonbinary_w5_df_names, 5)

# width: 10
repna_nonbinary_w10_df_dict = {}
repna_nonbinary_w10_df_names = ['IBS_repna_nonbinary_w10', "lupus_repna_nonbinary_w10", "rheum_arthr_repna_nonbinary_w10", 
                          "fibromyalgia_repna_nonbinary_w10", "dysautonomia_repna_nonbinary_w10", "psor_arthr_repna_nonbinary_w10", 
                          "arthr_repna_nonbinary_w10", "psor_repna_nonbinary_w10", "raynauds_repna_nonbinary_w10"]
each_cond_dataset(repna_nonbinary_df_dict, repna_nonbinary_df_names, repna_nonbinary_w10_df_dict, repna_nonbinary_w10_df_names, 10)

Lists of all the dictionaries that contain usable dataframes

In [34]:
# SHORT VERSION if you only ran the code to create the dataframes that drop the NaN values
good_df_dicts = [dropna_binary_w1_df_dict, dropna_binary_w3_df_dict, dropna_binary_w5_df_dict, dropna_binary_w10_df_dict,
               dropna_nonbinary_w1_df_dict, dropna_nonbinary_w3_df_dict, dropna_nonbinary_w5_df_dict, dropna_nonbinary_w10_df_dict]
# And a list of all the names of the dictionaries that are within the above dictionaries
good_df_names = [dropna_binary_w1_df_names, dropna_binary_w3_df_names, dropna_binary_w5_df_names, dropna_binary_w10_df_names,
               dropna_nonbinary_w1_df_names, dropna_nonbinary_w3_df_names, dropna_nonbinary_w5_df_names, dropna_nonbinary_w10_df_names]

In [ ]:
# COMPLETE VERSION
good_df_dicts = [dropna_binary_w1_df_dict, dropna_binary_w3_df_dict, dropna_binary_w5_df_dict, dropna_binary_w10_df_dict,
               dropna_nonbinary_w1_df_dict, dropna_nonbinary_w3_df_dict, dropna_nonbinary_w5_df_dict, dropna_nonbinary_w10_df_dict,
               repna_binary_w1_df_dict, repna_binary_w3_df_dict, repna_binary_w5_df_dict, repna_binary_w10_df_dict,
               repna_nonbinary_w1_df_dict, repna_nonbinary_w3_df_dict, repna_nonbinary_w5_df_dict, repna_nonbinary_w10_df_dict]
# And a list of all the names of the dictionaries that are within the above dictionaries
good_df_names = [dropna_binary_w1_df_names, dropna_binary_w3_df_names, dropna_binary_w5_df_names, dropna_binary_w10_df_names,
               dropna_nonbinary_w1_df_names, dropna_nonbinary_w3_df_names, dropna_nonbinary_w5_df_names, dropna_nonbinary_w10_df_names,
               repna_binary_w1_df_names, repna_binary_w3_df_names, repna_binary_w5_df_names, repna_binary_w10_df_names,
               repna_nonbinary_w1_df_names, repna_nonbinary_w3_df_names, repna_nonbinary_w5_df_names, repna_nonbinary_w10_df_names]

Store the dataframes (from the above dictionaries) in a file

In [46]:
# Keep track of the names of the new files
data_filenames = []

for i, df_dict in enumerate(good_df_dicts):
    for df_name in good_df_names[i]:
        this_df = df_dict[df_name]
        filename = "%s.pickle" %df_name
        
        with open("training_data/%s"%filename, 'wb') as handle:
            pickle.dump(this_df, handle, protocol=pickle.HIGHEST_PROTOCOL)
        data_filenames.append(filename)
        
# save the names of the data files to a pickle file (so they are accessible from other modules)
with open("trainingdata_filenames.pickle", 'wb') as handle:
            pickle.dump(data_filenames, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [36]:
# TEST####
X = dropna_binary_w5_df_dict[lupus_dropna_binary_w5].iloc[:,:-1]
print("x",X.shape)
# num columns should be:  26*5+1
y = dropna_binary_w5_df_dict[lupus_dropna_binary_w5].iloc[:,-1:]
print("y",y.shape)
# num columns should be:  26*5+1

NameError: name 'lupus_dropna_binary_w5' is not defined

### Remove rows that do not predict a flare up

This was recommended by a mentor, but it does not yield useful results, so I will skip this step for now

### Functions to separate X_train, X_test, y_train, y_test and normalize data

X: array of shape (n_samples, n_features)

y: array of shape (n_samples,)

In [47]:
def x_y_train_test(df):
    """
    (pd.DataFrame) -> dict
    
    Returns a dictionary that contains 5 data structures: X (pd.DataFrame), X_train, X_test, y_train, y_test (np.ndarray)
    
    """
    
    #X, y
    X = df.iloc[:,:-1]
    y = df.iloc[:,-1:]
    
    # train, test
    n_samples = X.shape[0]
    train_samples = int(0.8*n_samples) # Keep 80% of the sample for training
    
    X_train = X.iloc[:train_samples,:]
    X_test = X.iloc[train_samples:,:]
    y_train = y.iloc[:train_samples,:]
    y_test = y.iloc[train_samples:,:]
    
    dict_toreturn = {"X": X, "X_train": X_train, "X_test": X_test, "y_train": y_train, "y_test": y_test}
    
    return dict_toreturn

In [48]:
def normalize(scaler_str, X_train, X_test):
    """
    (str, np.ndarray, np.ndarray) -> dict
    
    Returns a dictionary that contains 2 arrays (X_train_norm and X_test_norm) normalized using scaler.
    """
    
    # Define scaler
    if scaler_str == "MinMaxScaler":
        scaler = preprocessing.MinMaxScaler()
    elif scaler_str == "StandardScaler":
        scaler = preprocessing.StandardScaler()
    
    # Fit scaler to train data
    scaler.fit(X_train)
    
    # Transform data
    X_train_norm = scaler.transform(X_train)
    X_test_norm = scaler.transform(X_test)
    
    dict_toreturn = {"X_train_norm": X_train_norm, "X_test_norm": X_test_norm}
    
    return dict_toreturn

## Model

Save the weight values (in a dataframe) to a pickle file. 

Save the test predictions (in a dataframe) to a pickle file.

This will make visualization easier.

_The name of the scaler can be changed_

In [49]:
# Keep track of the filenames that I use
weight_filenames = []
predictions_filenames = []

# Iterate through all the DataFrames that contain usable data
for i, df_dict in enumerate(good_df_dicts):
    for j,df_name in enumerate(good_df_names[i]):
        # get dataframe
        this_df = df_dict[df_name]
        # separate x and y, train and test
        xy_dict = x_y_train_test(this_df)
        X = xy_dict["X"]
        X_train = xy_dict["X_train"]
        X_test = xy_dict["X_test"]
        y_train = xy_dict["y_train"]
        y_test = xy_dict["y_test"]
        
        # normalize
        try:
            x_norm_dict = normalize("MinMaxScaler", X_train, X_test) # Name of scaler can be changed
            X_train = x_norm_dict["X_train_norm"]
            X_test = x_norm_dict["X_test_norm"]
        except (ValueError): ### I know this is cheating, but the problem is that some dataframes are empty. I'll try to fix it later.
            print("Error with:", df_name)
            continue
        
        # Simple linear regression
        regr = linear_model.LinearRegression().fit(X_train, y_train)
        
        # Get the coefficient of determination R^2 of the prediction
        # R2_score = regr.score(X_train, y_train)
        # print("R2 score:", R2_score)
        
        # See the weights that the model assigned to the input features
        weights = (regr.coef_)
        column_names = list(X.columns)
        weights_df = pd.DataFrame(data=weights, columns=column_names)
        weigths_ordered = weights_df.sort_values(by=0, axis='columns')
        # Write the weights DataFrame to a Pickle file
        filename = "weights/%s_WEIGHTS.pickle" %df_name
        with open(filename, 'wb') as handle:
            pickle.dump(weigths_ordered, handle, protocol=pickle.HIGHEST_PROTOCOL)
        weight_filenames.append(filename)
            
        # Test the model by making predictions
        y_pred = regr.predict(X_test)
        predictions = pd.DataFrame(data=y_pred,columns=["Pred"])
        y_test = y_test.reset_index()
        predictions = pd.concat([predictions, y_test], axis=1)
        predictions = predictions.drop("index", axis=1)
        # Write the predictions DataFrame to a Pickle file
        filename = "predictions/%s_PREDICTIONS.pickle" %df_name
        with open(filename, 'wb') as handle:
            pickle.dump(predictions, handle, protocol=pickle.HIGHEST_PROTOCOL)
        predictions_filenames.append(filename)

Error with: psor_arthr_dropna_binary_w1
Error with: psor_arthr_dropna_binary_w3
Error with: rheum_arthr_dropna_binary_w5
Error with: psor_arthr_dropna_binary_w5
Error with: rheum_arthr_dropna_binary_w10
Error with: dysautonomia_dropna_binary_w10
Error with: psor_arthr_dropna_binary_w10
Error with: arthr_dropna_binary_w10
Error with: psor_arthr_dropna_nonbinary_w1
Error with: psor_arthr_dropna_nonbinary_w3
Error with: rheum_arthr_dropna_nonbinary_w5
Error with: psor_arthr_dropna_nonbinary_w5
Error with: rheum_arthr_dropna_nonbinary_w10
Error with: dysautonomia_dropna_nonbinary_w10
Error with: psor_arthr_dropna_nonbinary_w10
Error with: arthr_dropna_nonbinary_w10


## Visualizations

### Weights

For each condition, create a DataFrame that contains all the weight values assigned to each feature at each training trial

In [50]:
# Make one big dataframe with the weight values of all of the features for all of the different training trials for each condition
all_weights_df_dict = {}
all_weights_df_names = ["IBS_all_weights_df", "lupus_all_weights_df", "fibromyalgia_all_weights_df","dysautonomia_all_weights_df",
                        "rheum_arthr_all_weights_df", "arthritis_all_weights_df", "psoriasis_all_weights_df", "raynauds_all_weights_df"]
# a dictionary that maps the name of an all_weights_df to the names of the files that contain this condition's weight info
cond_weight_filenames = {"IBS_all_weights_df":["IBS_dropna_binary_w1_WEIGHTS.pickle", "IBS_dropna_binary_w3_WEIGHTS.pickle", "IBS_dropna_binary_w5_WEIGHTS.pickle", "IBS_dropna_binary_w10_WEIGHTS.pickle","IBS_dropna_nonbinary_w1_WEIGHTS.pickle", "IBS_dropna_nonbinary_w3_WEIGHTS.pickle", "IBS_dropna_nonbinary_w5_WEIGHTS.pickle", "IBS_dropna_nonbinary_w10_WEIGHTS.pickle"], 
                         "lupus_all_weights_df":["lupus_dropna_binary_w1_WEIGHTS.pickle","lupus_dropna_binary_w3_WEIGHTS.pickle","lupus_dropna_binary_w5_WEIGHTS.pickle","lupus_dropna_binary_w10_WEIGHTS.pickle","lupus_dropna_nonbinary_w1_WEIGHTS.pickle", "lupus_dropna_nonbinary_w3_WEIGHTS.pickle", "lupus_dropna_nonbinary_w5_WEIGHTS.pickle", "lupus_dropna_nonbinary_w10_WEIGHTS.pickle"], 
                         "fibromyalgia_all_weights_df":["fibromyalgia_dropna_binary_w1_WEIGHTS.pickle","fibromyalgia_dropna_binary_w3_WEIGHTS.pickle","fibromyalgia_dropna_binary_w5_WEIGHTS.pickle","fibromyalgia_dropna_binary_w10_WEIGHTS.pickle","fibromyalgia_dropna_nonbinary_w1_WEIGHTS.pickle", "fibromyalgia_dropna_nonbinary_w3_WEIGHTS.pickle", "fibromyalgia_dropna_nonbinary_w5_WEIGHTS.pickle", "fibromyalgia_dropna_nonbinary_w10_WEIGHTS.pickle"],
                         "dysautonomia_all_weights_df":["dysautonomia_dropna_binary_w1_WEIGHTS.pickle","dysautonomia_dropna_binary_w3_WEIGHTS.pickle","dysautonomia_dropna_binary_w5_WEIGHTS.pickle","dysautonomia_dropna_nonbinary_w1_WEIGHTS.pickle","dysautonomia_dropna_nonbinary_w3_WEIGHTS.pickle","dysautonomia_dropna_nonbinary_w5_WEIGHTS.pickle"], 
                         "rheum_arthr_all_weights_df":["rheum_arthr_dropna_binary_w1_WEIGHTS.pickle","rheum_arthr_dropna_binary_w3_WEIGHTS.pickle", "rheum_arthr_dropna_nonbinary_w1_WEIGHTS.pickle","rheum_arthr_dropna_nonbinary_w1_WEIGHTS.pickle"],
                         "arthritis_all_weights_df":["arthr_dropna_binary_w1_WEIGHTS.pickle","arthr_dropna_binary_w3_WEIGHTS.pickle","arthr_dropna_binary_w5_WEIGHTS.pickle","arthr_dropna_nonbinary_w1_WEIGHTS.pickle","arthr_dropna_nonbinary_w3_WEIGHTS.pickle","arthr_dropna_nonbinary_w5_WEIGHTS.pickle"], 
                         "psoriasis_all_weights_df":["psor_dropna_binary_w1_WEIGHTS.pickle","psor_dropna_binary_w3_WEIGHTS.pickle","psor_dropna_binary_w5_WEIGHTS.pickle","psor_dropna_binary_w10_WEIGHTS.pickle","psor_dropna_nonbinary_w1_WEIGHTS.pickle","psor_dropna_nonbinary_w3_WEIGHTS.pickle","psor_dropna_nonbinary_w5_WEIGHTS.pickle","psor_dropna_nonbinary_w10_WEIGHTS.pickle"], 
                         "raynauds_all_weights_df":["raynauds_dropna_binary_w1_WEIGHTS.pickle","raynauds_dropna_binary_w3_WEIGHTS.pickle","raynauds_dropna_binary_w5_WEIGHTS.pickle","raynauds_dropna_binary_w10_WEIGHTS.pickle","raynauds_dropna_nonbinary_w1_WEIGHTS.pickle","raynauds_dropna_nonbinary_w3_WEIGHTS.pickle","raynauds_dropna_nonbinary_w5_WEIGHTS.pickle","raynauds_dropna_nonbinary_w10_WEIGHTS.pickle"]}
all_weights_df_filenames = []
for df_name in all_weights_df_names:
    all_weights_df = pd.DataFrame()
    for filename in cond_weight_filenames[df_name]:
        one_weight_df = pickle.load(open(filename, "rb" ))
        all_weights_df = pd.concat([all_weights_df, one_weight_df])
        all_weights_df_dict[df_name]=all_weights_df
    # save the dataframe containing all weights for this condition to a pickle file
    fname = "%s.pickle" %df_name
    with open(fname, 'wb') as handle:
        pickle.dump(all_weights_df, handle, protocol=pickle.HIGHEST_PROTOCOL)
    all_weights_df_filenames.append(fname)


To see which features play a role in the prediction and which do not, compute the average of the absolute weights for each feature for each condition

In [51]:
def abs_weight_average(weight_df_filename):
    """
    (str)->str
    
    weight_df_filename is the name of a pickle file that stores a pd.DataFrame.
    The function creates a DataFrame that contains the same columns as the df in weight_df_filename, but contains only one row.
    The values in this row are the average value of the absolute weights of each feature.
    This DataFrame is saved to a pickle file.
    The function returns the name of this file.
    """
    
    all_weights_df = pickle.load(open(weight_df_filename, "rb" ))
    all_abs_weights_df = all_weights_df.abs()
    # compute the average weight value for each feature
    weights_average_df=pd.DataFrame(index=[0])
    for col in all_abs_weights_df.columns:
        average = all_abs_weights_df.loc[:,col].mean()
        weights_average_df.loc[0,col]=average
    # sort features in ascending order of weight values
    weights_average_df = weights_average_df.sort_values(by=0, axis=1)
    # write to a pickle file
    fname = weight_df_filename.strip("_all_weights_df.pickle")
    fname="weights/%s_average_weight_values.pickle"%fname
    with open(fname, 'wb') as handle:
        pickle.dump(weights_average_df, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    return fname

_The visualization is not working properly, i'll have to fix it_

In [52]:
def visualize_weights(weights_average_df_filename, num_features):
    """
    (str, int) -> list
    
    weights_average_df_filename is the name of a pickle file that contains a pd.DataFrame.
    Using that DataFrame, the function creates two graphs:
        Most weighted features: bar graph showing the num_features most highly weighted features
        Least weighted features: bar graph showing the num_features lest weighted features
    These graphs are saved to a png file in the weight_plots folder.
    Returns a list that contains two dictionaries: the dict of the num_features most highly weighted features 
    and the dict of the num_features least weighted features (mapping to their average absolute weight values).
    """
    
    weights_average_df = pickle.load(open(weights_average_df_filename, "rb" ))
    # Make sure that weights_average_df is ordered in ascending order of weight values
    weights_average_df = weights_average_df.sort_values(by=0, axis=1)
    
    # Keep the highest weight values
    high_weights_df = weights_average_df.iloc[:,-num_features:]
    high_weights_dict = high_weights_df.to_dict(orient="list")
    for feature in high_weights_dict:
        high_weights_dict[feature]=high_weights_dict[feature][0]
    # plot highest weights
    high_weights_name = high_weights_dict.keys()
    high_weights_val = high_weights_dict.values()
    plt.bar(high_weights_name, high_weights_val, align="center")
    plt.xticks(rotation=45)
    title = weights_average_df_filename.replace("_average_weight_values.pickle","_highest_weights")
    plt.title(title)
    plt.savefig("weight_plots/%s.png"%title)
    plt.clf()
    
    # Keep the smallest weight values
    small_weights_df = weights_average_df.iloc[:,:-num_features]
    small_weights_dict = small_weights_df.to_dict(orient="list")
    for feature in small_weights_dict:
        small_weights_dict[feature]=small_weights_dict[feature][0]
    # plot smallest weights
    small_weights_name = small_weights_dict.keys()
    small_weights_val = small_weights_dict.values()
    plt.bar(small_weights_name, small_weights_val, align="center")
    plt.xticks(rotation=45)
    title = weights_average_df_filename.replace("_average_weight_values.pickle","_smallest_weights")
    plt.title(title)
    plt.savefig("weight_plots/%s.png"%title)
    plt.clf()
    
    # retun those dictionaries
    list_of_dict = [high_weights_dict,small_weights_dict]
    return list_of_dict

In [53]:
# Run the visualization functions
for weight_df_filename in all_weights_df_filenames:
    fname=abs_weight_average(weight_df_filename)
    visualize_weights(fname,8)

<Figure size 432x288 with 0 Axes>